In [1]:
import os
import re
import json
import pandas as pd


In [35]:
data_root = 'raw_data'
data_files = [f'{journal}_journals.csv'for journal in ['ce', 'di', 'wu', 'yao', 'yaoxi']]


In [43]:
split_pattern = re.compile(r'[;,/\s+]') # 关键词间可能的 分隔符
colon_pattern = re.compile(r'[:：]') # 中英文模式下的 冒号
space_pattern = re.compile(r'\s+')
zh_pattern = re.compile(u'[\u4e00-\u9fa5]+')
def split_keywords(keywords: str):
    '''str to list，去掉“关键词”、各种分隔符和空格'''
    # 去掉“关键词”字眼
    split_colon = colon_pattern.split(keywords)[-1]
    split_res = split_pattern.split(split_colon)
    keyword_list = []
    for keyword in split_res:
        keyword = keyword.strip()
        if len(keyword) > 0:
            keyword_list.append(keyword)
            
    return keyword_list

def match_keywords(keyword_list, text, precise_mode=True):
    '''从 keyword_list 中找到能在 text 中出现的 keyword，组成新的 keyword_list'''
    matched_list = []
    text = text.strip()
    for keyword in keyword_list:
        keyword = keyword.strip()
        keyword_len = len(keyword)
        if keyword_len < 2: continue # 只考虑字数大于等于 2 的关键词
        if keyword in text:
            matched_list.append(keyword)
        else:
            if not precise_mode: # 被严格匹配模式
                
                for start_ind in range(len(text)-keyword_len+2):
                    if keyword[0] != text[start_ind]: continue # 先保证第一字匹配上
                    for candidate_len in range(keyword_len-2, keyword_len+3): # 每个关键词的长度都大于 2
                        # 至少第一字和最后一个字是匹配上的
                        end_ind = start_ind + candidate_len - 1
                        end_ind = min(len(text)-1, end_ind)
                        if keyword[-1] == text[end_ind]:
                            matched_keyword = text[start_ind: end_ind+1]
                            print(f'找到了：{keyword}->{matched_keyword}, {keyword_list}')
                            if not any([matched_keyword in kw for kw in matched_list]):
                                matched_list.append(matched_keyword)
                            else:
                                print('但存在类似的')
                            break
                           
    return matched_list

def preprocess(data_path, save_json_path=None, file_mode='w'):
    
    data_frame = pd.read_csv(data_path, usecols=['title', 'keywords'])
    with open(save_json_path, file_mode, encoding='utf-8') as writer:
        for index, row in data_frame.iterrows():
            title, keywords = row['title'], row['keywords']
            title = space_pattern.sub('', title)
            # 只针对中文标题
            if not zh_pattern.search(title):
                print(title)
                continue
            keyword_list = split_keywords(keywords)
            keyword_matched = match_keywords(keyword_list, title, precise_mode=False)
            one_dict = {'title': title, 'keywords_matched': keyword_matched, 'keywords': keyword_list}
            writer.write(json.dumps(one_dict, ensure_ascii=False))
            writer.write('\n')

In [44]:
for i, data_f in enumerate(data_files):
    data_path = os.path.join(data_root, data_f)
    #data_frame = pd.read_csv(data_path, usecols=['title', 'keywords'])
    file_mode = 'w' if i == 0 else 'a'
    preprocess(data_path, 'data/journals/preprocessed_data.jl', file_mode= file_mode)
    

找到了：冰川泥石流->冰川流, ['智能监测', '天空地协同', '冰川流域', '冰川泥石流', '任务驱动', '知识引导']
但存在类似的
找到了：时空大数据地图表达->时空数据地图表达, ['时空大数据地图表达', '地图基础理论数学化', '地图设计定量化', '地图表达自适化', '质量预测模型化', '制图应用泛在化']
找到了：遥感智能解译->遥感影像智能解译, ['遥感智能解译', '样本库', '多源遥感影像', '数据模型', '深度学习']
找到了：标线分类->标线自动分类, ['车载激光点云', '标线分类', '图结构', '注意力机制', '标线矢量化']
找到了：三维地表形变->三维同震地表形变, ['干涉合成孔径雷达', '三维地表形变', 'SM-VCE', '窗口优化', '迭代加权最小二乘方法']
找到了：矢量数据溯源->矢量地理信息溯源, ['矢量数据溯源', '区块链', '溯源粒度', 'MPT结构', '溯源验证']
找到了：空间认知->空间形状认知, ['空间认知', '形状编码', '深度学习', '自编码器', 'sequence-to-sequence模型']
找到了：结构张量->结合张量, ['各向异性滤波', '多模态图像', '结构张量', '相似度准则', '图像配准']
找到了：业务化测绘->业务化地形测绘, ['SAR测地学', '雷达干涉', '业务化测绘', '检校', '平面及高程精度']
找到了：边界入射信号->边界信号, ['GNSS', '三维水汽', '层析', '边界入射信号', '垂直约束', '无线电探空数据']
找到了：珠峰高程测量->珠峰测量, ['珠峰高程测量', 'GNSS', '航空重力', '大地水准面', '国际高程参考系统']
找到了：多参数优化->多参数协同优化, ['反距离加权插值(IDW)', '空间邻近度', '各向异性', '多参数优化', '粒子群算法']
找到了：倾斜立体影像->倾斜影像, ['倾斜立体影像', '卷积神经网络', '仿射不变特征', '深度学习', '影像匹配']
找到了：侧扫声呐海底沉船->侧扫声呐识别沉船, ['侧扫声呐海底沉船', '图像识别', '迁移学习', '卷积神经网络', 'VGG-16']
找到了：影像匹配->影